In [2]:
from google.colab import files
uploaded = files.upload()

Saving Dataset030921.csv to Dataset030921.csv
Saving slang.csv to slang.csv


# Sett Awal

In [ ]:
!pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyswarms

In [ ]:
import re
import nltk
import string
import numpy as np
import pandas as pd
import seaborn as sns
import pyswarms as ps
import matplotlib.pyplot as plt
nltk.download('punkt')
nltk.download('stopwords')
from time import time
from random import random
from random import uniform
from tabulate import tabulate
from numpy.random import rand
from numpy.random import randint
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
class Particle:
    def __init__(self, x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i<self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i.copy()
            self.err_best_i=self.err_i
                    
    # update new particle velocity
    def update_velocity(self,pos_best_g):
        # w=0.9       # constant inertia weight (how much to weigh the previous velocity)
        # c1=0.5        # cognative constant
        # c2=0.3        # social constant
        
        for i in range(0,num_dimensions):
            r1=random()
            r2=random()
            
            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]
            
            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i]<bounds[i][0]:
                self.position_i[i]=bounds[i][0]
        
        
def minimize(costFunc, x0, bounds, num_particles, maxiter, verbose=False):
    global num_dimensions
    global w, c1, c2

    w=0.9
    c1=0.5
    c2=0.5
    num_dimensions=len(x0)
    err_best_g=-1                   # best error for group
    pos_best_g=[]                   # best position for group

    # establish the swarm
    swarm=[]
    for i in range(0,num_particles):
        swarm.append(Particle(x0))

    # begin optimization loop
    i=0
    while i<maxiter:
        x = np.array(pos_best_g)
        sel = x>0.5
        tot = sel.sum()
        pos_best_g = list(x)
        if verbose: print(f'iter: {i:>4d}, sel : {tot:6} best solution: {err_best_g:10.6f} , posisition: {pos_best_g}')
            
        # cycle through particles in swarm and evaluate fitness
        for j in range(0,num_particles):
            swarm[j].evaluate(costFunc)

            # determine if current particle is the best (globally)
            if swarm[j].err_i<err_best_g or err_best_g==-1:
                pos_best_g=list(swarm[j].position_i)
                err_best_g=float(swarm[j].err_i)
        
        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
            swarm[j].update_velocity(pos_best_g)
            swarm[j].update_position(bounds)
        i+=1

    # print final results
    if verbose:
        print('\nFINAL SOLUTION:')
        print(f'   > {i:>4d}')
        print(f'   > {pos_best_g}')
        print(f'   > {err_best_g}\n')

    return err_best_g, pos_best_g


def print_df(dframe):
    print(tabulate(dframe, headers='keys', tablefmt='psql', showindex=True))

# tournament selection
def selection(pop, scores, k=3):
  # first random selection
  selection_ix = randint(len(pop))
  for ix in randint(0, len(pop), k-1):
    # check if better (e.g. perform a tournament)
    if scores[ix] < scores[selection_ix]:
      selection_ix = ix
  return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]
 
# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]
 
# genetic algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
  # initial population of random bitstring
  pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
  # keep track of best solution
  best, best_eval = 0, objective(pop[0])
  # enumerate generations
  for gen in range(n_iter):
    # evaluate all candidates in the population
    scores = [objective(c) for c in pop]
    # check for new best solution
    for i in range(n_pop):
      if scores[i] < best_eval:
        best, best_eval = pop[i], scores[i]
        print(">%d, new best %.3f = f(%s)" % (gen, scores[i] , pop[i]))
    # select parents
    selected = [selection(pop, scores) for _ in range(n_pop)]
    # create the next generation
    children = list()
    for i in range(0, n_pop, 2):
      # get selected parents in pairs
      p1, p2 = selected[i], selected[i+1]
      # crossover and mutation
      for c in crossover(p1, p2, r_cross):
        # mutation
        mutation(c, r_mut)
        # store for next generation
        children.append(c)
    # replace population
    pop = children
  return [best, best_eval]

def fitfuncGA(x):
  fitn = 0
  sel = np.array(x)==1
  tot = sum(sel)
  posisi.append(sel)
  total.append(tot)
  accuracy = cross_val_score(model, text_tw[:,sel], label_tw, scoring='accuracy', cv=10, n_jobs=-1).mean()
  acc.append(accuracy)
  fitn = alpha * (1-accuracy) + (1 - alpha) * (tot / feature_count)
  fitness.append(fitn)
  return fitn

In [ ]:
df = pd.read_csv('Dataset030921.csv',sep=";", encoding='cp1252')
df

In [ ]:
kamus_normalisasi = pd.read_csv("slang.csv",sep=";", encoding='cp1252')
normalisasi_dict = {}

for index, row in kamus_normalisasi.iterrows():
  if row[0] not in normalisasi_dict:
    normalisasi_dict[row[0]] = row[1]

stop_factory = StopWordRemoverFactory()
stopwordNLTK = stopwords.words('indonesian')
stopwordSastrawi = stop_factory.get_stop_words()
more_stopword = ['twitwar', 'repost', 'rt', 'are', 'deh','kah', 
                 'htg', 'wk','aa','cc','sih', 'ayo', 'si','dsb',
                 'loh','an','ma','dimana','eh','aduh','lho','oh',
                 'ii','yah','sok','ah','of','ei','er','ie','prof',
                 'gus','om','fess','do']
filterStopword = set(stopwordSastrawi + stopwordNLTK + more_stopword)

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

tweetStem = []
tweetJoin = []

for indeks,tweets in enumerate(df['tweet']):
  removed = []
  tweet = []
  # case folding
  tweets = tweets.lower()
  # cleaning
  tweets = ' '.join(re.sub('2021|21'," duaribuduasatu",tweets).split())
  tweets = ' '.join(re.sub('2022'," duaribuduadua",tweets).split())
  tweets = ' '.join(re.sub('30'," tigapuluh",tweets).split())
  tweets = ' '.join(re.sub('4|a{3,}',"a",tweets).split())
  parnwk = ("wk{2,}|wkw{2,}|y{2,}|ww+|kwk+|\d{5,}|haha+|z{2,}|xix+|xixi+|ixi+|xx+|wq+|hehe+|waka(ka)+")
  tweets = ' '.join(re.sub(parnwk," ",tweets).split())
  tweets = ' '.join(re.sub('3|ee+',"e",tweets).split())
  tweets = ' '.join(re.sub('h+',"h",tweets).split())
  tweets = ' '.join(re.sub('1|i{3,}',"i",tweets).split())
  tweets = ' '.join(re.sub('u{3,}',"u",tweets).split())
  tweets = ' '.join(re.sub('k+',"k",tweets).split())
  tweets = ' '.join(re.sub('s+',"s",tweets).split())
  tweets = ' '.join(re.sub('o+',"o",tweets).split())
  tweets = ' '.join(re.sub('r+',"r",tweets).split())
  tweets = ' '.join(re.sub('g{3,}',"g",tweets).split())
  tweets = ' '.join(re.sub("([^a-zA-Z])"," ",tweets).split())
  # normalisasi
  tweets = " ".join(normalisasi_dict.get(word, word) for word in tweets.split(' '))
  # stopword
  for t in tweets.split(' '):
    if t not in filterStopword:
      removed.append(t)
  for tweets in removed:
    tweet.append(tweets)
  tweets = ' '.join(tweet)
  # stemmer
  tweets = stemmer.stem(tweets)
  tweetStem.append(tweets.split())
  tweetJoin.append(tweets)

data = {'tweet': tweetStem, 'tweet join':tweetJoin, 'label':df['label']}
dataTweet = pd.DataFrame(data)
dataTweet

,tweet,tweet join,label
0,"[mahasiswa, desak, sah, permendikbud, ristek, ...",mahasiswa desak sah permendikbud ristek ppks u...,0
1,"[permendikbud, cegah, keras, seksual]",permendikbud cegah keras seksual,0
2,"[kabar, gembira, permendikbud, cegah, tangan, ...",kabar gembira permendikbud cegah tangan keras ...,0
3,"[alhamdulilah, permendikbud, cegah, tangan, ke...",alhamdulilah permendikbud cegah tangan keras s...,0
4,"[gerak, komitmen, juang, adil, korban, keras, ...",gerak komitmen juang adil korban keras seksual...,0
...,...,...,...
15632,"[apresiasi, kemendikbud, permendikbud, ristek,...",apresiasi kemendikbud permendikbud ristek no t...,0
15633,"[narasi, kemarin, ramai, tolak, permendikbud, ...",narasi kemarin ramai tolak permendikbud atur p...,0
15634,"[legal, zina, kakak, tolak, perkara, permendik...",legal zina kakak tolak perkara permendikbud ke...,2
15635,"[duaribuduasatu, ruu, tahap, prolegnas, dpr, b...",duaribuduasatu ruu tahap prolegnas dpr beda da...,0


In [ ]:
dataTweet.label.value_counts()

0    7684
1    4892
2    3061
Name: label, dtype: int64

In [ ]:
tf=TfidfVectorizer()
text_tw  = tf.fit_transform(dataTweet['tweet join'])
label_tw = df['label']
params    = [x for x in range(1,11)]
feature_names = tf.get_feature_names_out()
feature_count = len(feature_names)
print(feature_count)

13469


# Kernel Sigmoid

In [ ]:
model = SVC(kernel='sigmoid', gamma=0.3, C=0.5, decision_function_shape='ovo')
scoring = {'accuracy': make_scorer(metrics.accuracy_score),
           'precision': make_scorer(metrics.precision_score, average = 'weighted'),
           'recall': make_scorer(metrics.recall_score, average = 'weighted'),
           'f1_weighted': make_scorer(metrics.f1_score, average = 'weighted')}
t = time()
hasil= cross_validate(model,text_tw, label_tw, scoring=scoring,
                       cv=10, return_train_score=True)
test_time = time() - t
d = pd.DataFrame(params)
d['test acc']   = hasil['test_accuracy']
d['test pre']   = hasil['test_precision']
d['test rec']   = hasil['test_recall']
d['test f1 ']   = hasil['test_f1_weighted']
d['train acc']  = hasil['train_accuracy']
d['train pre']  = hasil['train_precision']
d['train rec']  = hasil['train_recall']
d['train f1 ']  = hasil['train_f1_weighted']
d['time']       = (hasil['fit_time'] + hasil['score_time'])
print_df(d)
y_pred = cross_val_predict(model, text_tw, label_tw, cv=10, n_jobs=-1)
print("\nMNB Accuracy   : %0.3f" % metrics.accuracy_score(label_tw, y_pred))
print("MNB Weighted Precision  : %0.3f" % metrics.precision_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted Recall     : %0.3f" % metrics.recall_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted F1 Score   : %0.3f" % metrics.f1_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted time train : %0.3fs" % test_time)
print("MNB Macro Precision  : %0.3f" % metrics.precision_score(label_tw, y_pred, average='macro'))
print("MNB Macro Recall     : %0.3f" % metrics.recall_score(label_tw, y_pred, average='macro'))
print("MNB Macro F1 Score   : %0.3f" % metrics.f1_score(label_tw, y_pred, average='macro'))
print("MNB Macro time train : %0.3fs" % test_time)
print("\nconfusion matrix:")
print(metrics.confusion_matrix(label_tw, y_pred))
print("\nclassification report:")
print(metrics.classification_report(label_tw, y_pred,
                                    target_names=['Positive','Negative', 'Neutral']))

(ppos, pneg, pneu) = metrics.precision_score(label_tw, y_pred, average=None)
(rpos, rneg, rneu) = metrics.recall_score(label_tw, y_pred, average=None)
(fpos, fneg, fneu) = metrics.f1_score(label_tw, y_pred, average=None)
print("\n         precision  recall   f1")
print("positif    %0.3f" % ppos, "   %0.3f" % rpos, "  %0.3f" % fpos)
print("negatif    %0.3f" % pneg, "   %0.3f" % rneg, "  %0.3f" % fpos)
print("neutral    %0.3f" % pneu, "   %0.3f" % rneu, "  %0.3f" % fneu)
print("accuracy                    %0.3f \n"% metrics.accuracy_score(label_tw, y_pred))

+----+-----+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-----------+
|    |   0 |   test acc |   test pre |   test rec |   test f1  |   train acc |   train pre |   train rec |   train f1  |      time |
|----+-----+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-----------|
|  0 |   1 |   0.627238 |   0.701572 |   0.627238 |   0.561505 |    0.739928 |    0.807764 |    0.739928 |    0.715101 | 0.0220373 |
|  1 |   2 |   0.744246 |   0.807083 |   0.744246 |   0.68943  |    0.728487 |    0.798802 |    0.728487 |    0.701928 | 0.0273864 |
|  2 |   3 |   0.78133  |   0.831767 |   0.78133  |   0.752757 |    0.726071 |    0.796132 |    0.726071 |    0.697208 | 0.0208204 |
|  3 |   4 |   0.68798  |   0.753537 |   0.68798  |   0.651946 |    0.733888 |    0.799521 |    0.733888 |    0.706845 | 0.0199568 |
|  4 |   5 |   0.622123 |   0.705493 |   0.622123 |   0.563017 |    0

In [ ]:
from sklearn.model_selection import StratifiedKFold
import statistics

fold = StratifiedKFold(n_splits=10)

X = text_tw
y = label_tw

a = 1
accs = []
weigted = []
for train, test in fold.split(text_tw, label_tw):
  ypred = model.fit(X[train], y[train]).predict(X[test])
  print("Fold ", a)
  (ppos, pneg, pneu) = metrics.precision_score(y[test], ypred, average=None)
  (rpos, rneg, rneu) = metrics.recall_score(y[test], ypred, average=None)
  (fpos, fneg, fneu) = metrics.f1_score(y[test], ypred, average=None)
  weigted.append([metrics.accuracy_score(y[test], ypred), metrics.precision_score(y[test], ypred, average='weighted'),metrics.recall_score(y[test], ypred, average='weighted'),metrics.f1_score(y[test], ypred, average='weighted')])
  print("         precision  recall   f1")
  print("positif    %0.3f" % ppos, "   %0.3f" % rpos, "  %0.3f" % fpos)
  print("negatif    %0.3f" % pneg, "   %0.3f" % rneg, "  %0.3f" % fpos)
  print("neutral    %0.3f" % pneu, "   %0.3f" % rneu, "  %0.3f" % fneu)
  print("accuracy                    %0.3f \n"% metrics.accuracy_score(y[test], ypred))
  print(metrics.confusion_matrix(y[test], ypred))
  print("\n")
  a = a + 1
a = 1
print("         acc   prec   rec    f1")
wpres = []
wrecs = []
wf1ss = []
for acc, wpre, wrec, wf1s in weigted:
  accs.append(acc)
  wpres.append(wpre)
  wrecs.append(wrec)
  wf1ss.append(wf1s)
  print("Fold %2.0f" % a, "%2.3f" % acc, "%2.3f" % wpre, " %2.3f" % wrec, " %2.3f" % wf1s)
  a=a+1

print("\nstdev acc : %0.3f"% statistics.stdev(accs))
print("stdev pre : %0.3f"% statistics.stdev(wpres))
print("stdev rec : %0.3f"% statistics.stdev(wrecs))
print("stdev f1s : %0.3f"% statistics.stdev(wf1ss))

Fold  1
         precision  recall   f1
positif    0.592    0.967   0.734
negatif    0.762    0.440   0.734
neutral    0.880    0.072   0.133
accuracy                    0.627 

[[744  25   0]
 [271 215   3]
 [242  42  22]]


Fold  2
         precision  recall   f1
positif    0.676    0.987   0.803
negatif    0.913    0.769   0.803
neutral    0.967    0.095   0.173
accuracy                    0.744 

[[759  10   0]
 [112 376   1]
 [251  26  29]]


Fold  3
         precision  recall   f1
positif    0.704    0.986   0.821
negatif    0.951    0.787   0.821
neutral    0.963    0.258   0.407
accuracy                    0.781 

[[758   8   3]
 [104 385   0]
 [215  12  79]]


Fold  4
         precision  recall   f1
positif    0.631    0.953   0.759
negatif    0.838    0.573   0.759
neutral    0.926    0.206   0.337
accuracy                    0.688 

[[733  34   2]
 [206 280   3]
 [223  20  63]]


Fold  5
         precision  recall   f1
positif    0.580    0.975   0.727
negatif    0.821    0.

# Kernel RBF

In [3]:
model = SVC(kernel='RBF', gamma=0.3, C=0.5, decision_function_shape='ovo')
scoring = {'accuracy': make_scorer(metrics.accuracy_score),
           'precision': make_scorer(metrics.precision_score, average = 'weighted'),
           'recall': make_scorer(metrics.recall_score, average = 'weighted'),
           'f1_weighted': make_scorer(metrics.f1_score, average = 'weighted')}
t = time()
hasil= cross_validate(model,text_tw, label_tw, scoring=scoring,
                       cv=10, return_train_score=True)
test_time = time() - t
d = pd.DataFrame(params)
d['test acc']   = hasil['test_accuracy']
d['test pre']   = hasil['test_precision']
d['test rec']   = hasil['test_recall']
d['test f1 ']   = hasil['test_f1_weighted']
d['train acc']  = hasil['train_accuracy']
d['train pre']  = hasil['train_precision']
d['train rec']  = hasil['train_recall']
d['train f1 ']  = hasil['train_f1_weighted']
d['time']       = (hasil['fit_time'] + hasil['score_time'])
print_df(d)
y_pred = cross_val_predict(model, text_tw, label_tw, cv=10, n_jobs=-1)
print("\nMNB Accuracy   : %0.3f" % metrics.accuracy_score(label_tw, y_pred))
print("MNB Weighted Precision  : %0.3f" % metrics.precision_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted Recall     : %0.3f" % metrics.recall_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted F1 Score   : %0.3f" % metrics.f1_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted time train : %0.3fs" % test_time)
print("MNB Macro Precision  : %0.3f" % metrics.precision_score(label_tw, y_pred, average='macro'))
print("MNB Macro Recall     : %0.3f" % metrics.recall_score(label_tw, y_pred, average='macro'))
print("MNB Macro F1 Score   : %0.3f" % metrics.f1_score(label_tw, y_pred, average='macro'))
print("MNB Macro time train : %0.3fs" % test_time)
print("\nconfusion matrix:")
print(metrics.confusion_matrix(label_tw, y_pred))
print("\nclassification report:")
print(metrics.classification_report(label_tw, y_pred,
                                    target_names=['Positive','Negative', 'Neutral']))

(ppos, pneg, pneu) = metrics.precision_score(label_tw, y_pred, average=None)
(rpos, rneg, rneu) = metrics.recall_score(label_tw, y_pred, average=None)
(fpos, fneg, fneu) = metrics.f1_score(label_tw, y_pred, average=None)
print("\n         precision  recall   f1")
print("positif    %0.3f" % ppos, "   %0.3f" % rpos, "  %0.3f" % fpos)
print("negatif    %0.3f" % pneg, "   %0.3f" % rneg, "  %0.3f" % fpos)
print("neutral    %0.3f" % pneu, "   %0.3f" % rneu, "  %0.3f" % fneu)
print("accuracy                    %0.3f \n"% metrics.accuracy_score(label_tw, y_pred))

+----+-----+------------+------------+------------+------------+-------------+-------------+-------------+-------------+------------+
|    |   0 |   test acc |   test pre |   test rec |   test f1  |   train acc |   train pre |   train rec |   train f1  |       time |
|----+-----+------------+------------+------------+------------+-------------+-------------+-------------+-------------+------------|
|  0 |   1 |        nan |        nan |        nan |        nan |         nan |         nan |         nan |         nan | 0.00903034 |
|  1 |   2 |        nan |        nan |        nan |        nan |         nan |         nan |         nan |         nan | 0.00888753 |
|  2 |   3 |        nan |        nan |        nan |        nan |         nan |         nan |         nan |         nan | 0.00857854 |
|  3 |   4 |        nan |        nan |        nan |        nan |         nan |         nan |         nan |         nan | 0.0084734  |
|  4 |   5 |        nan |        nan |        nan |        nan

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py", line 342, in _sparse_fit
    kernel_type = self._sparse_kernels.index(kernel)
ValueError: 'RB

ValueError: ignored

In [ ]:
from sklearn.model_selection import StratifiedKFold
import statistics

fold = StratifiedKFold(n_splits=10)

X = text_tw
y = label_tw

a = 1
accs = []
weigted = []
for train, test in fold.split(text_tw, label_tw):
  ypred = model.fit(X[train], y[train]).predict(X[test])
  print("Fold ", a)
  (ppos, pneg, pneu) = metrics.precision_score(y[test], ypred, average=None)
  (rpos, rneg, rneu) = metrics.recall_score(y[test], ypred, average=None)
  (fpos, fneg, fneu) = metrics.f1_score(y[test], ypred, average=None)
  weigted.append([metrics.accuracy_score(y[test], ypred), metrics.precision_score(y[test], ypred, average='weighted'),metrics.recall_score(y[test], ypred, average='weighted'),metrics.f1_score(y[test], ypred, average='weighted')])
  print("         precision  recall   f1")
  print("positif    %0.3f" % ppos, "   %0.3f" % rpos, "  %0.3f" % fpos)
  print("negatif    %0.3f" % pneg, "   %0.3f" % rneg, "  %0.3f" % fpos)
  print("neutral    %0.3f" % pneu, "   %0.3f" % rneu, "  %0.3f" % fneu)
  print("accuracy                    %0.3f \n"% metrics.accuracy_score(y[test], ypred))
  print(metrics.confusion_matrix(y[test], ypred))
  print("\n")
  a = a + 1
a = 1
print("         acc   prec   rec    f1")
wpres = []
wrecs = []
wf1ss = []
for acc, wpre, wrec, wf1s in weigted:
  accs.append(acc)
  wpres.append(wpre)
  wrecs.append(wrec)
  wf1ss.append(wf1s)
  print("Fold %2.0f" % a, "%2.3f" % acc, "%2.3f" % wpre, " %2.3f" % wrec, " %2.3f" % wf1s)
  a=a+1

print("\nstdev acc : %0.3f"% statistics.stdev(accs))
print("stdev pre : %0.3f"% statistics.stdev(wpres))
print("stdev rec : %0.3f"% statistics.stdev(wrecs))
print("stdev f1s : %0.3f"% statistics.stdev(wf1ss))

# Kernel Polynomial

In [ ]:
model = SVC(kernel='sigmoid', gamma=0.3, C=0.5, decision_function_shape='ovo')
scoring = {'accuracy': make_scorer(metrics.accuracy_score),
           'precision': make_scorer(metrics.precision_score, average = 'weighted'),
           'recall': make_scorer(metrics.recall_score, average = 'weighted'),
           'f1_weighted': make_scorer(metrics.f1_score, average = 'weighted')}
t = time()
hasil= cross_validate(model,text_tw, label_tw, scoring=scoring,
                       cv=10, return_train_score=True)
test_time = time() - t
d = pd.DataFrame(params)
d['test acc']   = hasil['test_accuracy']
d['test pre']   = hasil['test_precision']
d['test rec']   = hasil['test_recall']
d['test f1 ']   = hasil['test_f1_weighted']
d['train acc']  = hasil['train_accuracy']
d['train pre']  = hasil['train_precision']
d['train rec']  = hasil['train_recall']
d['train f1 ']  = hasil['train_f1_weighted']
d['time']       = (hasil['fit_time'] + hasil['score_time'])
print_df(d)
y_pred = cross_val_predict(model, text_tw, label_tw, cv=10, n_jobs=-1)
print("\nMNB Accuracy   : %0.3f" % metrics.accuracy_score(label_tw, y_pred))
print("MNB Weighted Precision  : %0.3f" % metrics.precision_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted Recall     : %0.3f" % metrics.recall_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted F1 Score   : %0.3f" % metrics.f1_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted time train : %0.3fs" % test_time)
print("MNB Macro Precision  : %0.3f" % metrics.precision_score(label_tw, y_pred, average='macro'))
print("MNB Macro Recall     : %0.3f" % metrics.recall_score(label_tw, y_pred, average='macro'))
print("MNB Macro F1 Score   : %0.3f" % metrics.f1_score(label_tw, y_pred, average='macro'))
print("MNB Macro time train : %0.3fs" % test_time)
print("\nconfusion matrix:")
print(metrics.confusion_matrix(label_tw, y_pred))
print("\nclassification report:")
print(metrics.classification_report(label_tw, y_pred,
                                    target_names=['Positive','Negative', 'Neutral']))

(ppos, pneg, pneu) = metrics.precision_score(label_tw, y_pred, average=None)
(rpos, rneg, rneu) = metrics.recall_score(label_tw, y_pred, average=None)
(fpos, fneg, fneu) = metrics.f1_score(label_tw, y_pred, average=None)
print("\n         precision  recall   f1")
print("positif    %0.3f" % ppos, "   %0.3f" % rpos, "  %0.3f" % fpos)
print("negatif    %0.3f" % pneg, "   %0.3f" % rneg, "  %0.3f" % fpos)
print("neutral    %0.3f" % pneu, "   %0.3f" % rneu, "  %0.3f" % fneu)
print("accuracy                    %0.3f \n"% metrics.accuracy_score(label_tw, y_pred))

+----+-----+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-----------+
|    |   0 |   test acc |   test pre |   test rec |   test f1  |   train acc |   train pre |   train rec |   train f1  |      time |
|----+-----+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-----------|
|  0 |   1 |   0.627238 |   0.701572 |   0.627238 |   0.561505 |    0.739928 |    0.807764 |    0.739928 |    0.715101 | 0.0220373 |
|  1 |   2 |   0.744246 |   0.807083 |   0.744246 |   0.68943  |    0.728487 |    0.798802 |    0.728487 |    0.701928 | 0.0273864 |
|  2 |   3 |   0.78133  |   0.831767 |   0.78133  |   0.752757 |    0.726071 |    0.796132 |    0.726071 |    0.697208 | 0.0208204 |
|  3 |   4 |   0.68798  |   0.753537 |   0.68798  |   0.651946 |    0.733888 |    0.799521 |    0.733888 |    0.706845 | 0.0199568 |
|  4 |   5 |   0.622123 |   0.705493 |   0.622123 |   0.563017 |    0

In [ ]:
from sklearn.model_selection import StratifiedKFold
import statistics

fold = StratifiedKFold(n_splits=10)

X = text_tw
y = label_tw

a = 1
accs = []
weigted = []
for train, test in fold.split(text_tw, label_tw):
  ypred = model.fit(X[train], y[train]).predict(X[test])
  print("Fold ", a)
  (ppos, pneg, pneu) = metrics.precision_score(y[test], ypred, average=None)
  (rpos, rneg, rneu) = metrics.recall_score(y[test], ypred, average=None)
  (fpos, fneg, fneu) = metrics.f1_score(y[test], ypred, average=None)
  weigted.append([metrics.accuracy_score(y[test], ypred), metrics.precision_score(y[test], ypred, average='weighted'),metrics.recall_score(y[test], ypred, average='weighted'),metrics.f1_score(y[test], ypred, average='weighted')])
  print("         precision  recall   f1")
  print("positif    %0.3f" % ppos, "   %0.3f" % rpos, "  %0.3f" % fpos)
  print("negatif    %0.3f" % pneg, "   %0.3f" % rneg, "  %0.3f" % fpos)
  print("neutral    %0.3f" % pneu, "   %0.3f" % rneu, "  %0.3f" % fneu)
  print("accuracy                    %0.3f \n"% metrics.accuracy_score(y[test], ypred))
  print(metrics.confusion_matrix(y[test], ypred))
  print("\n")
  a = a + 1
a = 1
print("         acc   prec   rec    f1")
wpres = []
wrecs = []
wf1ss = []
for acc, wpre, wrec, wf1s in weigted:
  accs.append(acc)
  wpres.append(wpre)
  wrecs.append(wrec)
  wf1ss.append(wf1s)
  print("Fold %2.0f" % a, "%2.3f" % acc, "%2.3f" % wpre, " %2.3f" % wrec, " %2.3f" % wf1s)
  a=a+1

print("\nstdev acc : %0.3f"% statistics.stdev(accs))
print("stdev pre : %0.3f"% statistics.stdev(wpres))
print("stdev rec : %0.3f"% statistics.stdev(wrecs))
print("stdev f1s : %0.3f"% statistics.stdev(wf1ss))

Fold  1
         precision  recall   f1
positif    0.592    0.967   0.734
negatif    0.762    0.440   0.734
neutral    0.880    0.072   0.133
accuracy                    0.627 

[[744  25   0]
 [271 215   3]
 [242  42  22]]


Fold  2
         precision  recall   f1
positif    0.676    0.987   0.803
negatif    0.913    0.769   0.803
neutral    0.967    0.095   0.173
accuracy                    0.744 

[[759  10   0]
 [112 376   1]
 [251  26  29]]


Fold  3
         precision  recall   f1
positif    0.704    0.986   0.821
negatif    0.951    0.787   0.821
neutral    0.963    0.258   0.407
accuracy                    0.781 

[[758   8   3]
 [104 385   0]
 [215  12  79]]


Fold  4
         precision  recall   f1
positif    0.631    0.953   0.759
negatif    0.838    0.573   0.759
neutral    0.926    0.206   0.337
accuracy                    0.688 

[[733  34   2]
 [206 280   3]
 [223  20  63]]


Fold  5
         precision  recall   f1
positif    0.580    0.975   0.727
negatif    0.821    0.

# Kernel Linear

In [ ]:
model = SVC(kernel='sigmoid', gamma=0.3, C=0.5, decision_function_shape='ovo')
scoring = {'accuracy': make_scorer(metrics.accuracy_score),
           'precision': make_scorer(metrics.precision_score, average = 'weighted'),
           'recall': make_scorer(metrics.recall_score, average = 'weighted'),
           'f1_weighted': make_scorer(metrics.f1_score, average = 'weighted')}
t = time()
hasil= cross_validate(model,text_tw, label_tw, scoring=scoring,
                       cv=10, return_train_score=True)
test_time = time() - t
d = pd.DataFrame(params)
d['test acc']   = hasil['test_accuracy']
d['test pre']   = hasil['test_precision']
d['test rec']   = hasil['test_recall']
d['test f1 ']   = hasil['test_f1_weighted']
d['train acc']  = hasil['train_accuracy']
d['train pre']  = hasil['train_precision']
d['train rec']  = hasil['train_recall']
d['train f1 ']  = hasil['train_f1_weighted']
d['time']       = (hasil['fit_time'] + hasil['score_time'])
print_df(d)
y_pred = cross_val_predict(model, text_tw, label_tw, cv=10, n_jobs=-1)
print("\nMNB Accuracy   : %0.3f" % metrics.accuracy_score(label_tw, y_pred))
print("MNB Weighted Precision  : %0.3f" % metrics.precision_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted Recall     : %0.3f" % metrics.recall_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted F1 Score   : %0.3f" % metrics.f1_score(label_tw, y_pred, average='weighted'))
print("MNB Weighted time train : %0.3fs" % test_time)
print("MNB Macro Precision  : %0.3f" % metrics.precision_score(label_tw, y_pred, average='macro'))
print("MNB Macro Recall     : %0.3f" % metrics.recall_score(label_tw, y_pred, average='macro'))
print("MNB Macro F1 Score   : %0.3f" % metrics.f1_score(label_tw, y_pred, average='macro'))
print("MNB Macro time train : %0.3fs" % test_time)
print("\nconfusion matrix:")
print(metrics.confusion_matrix(label_tw, y_pred))
print("\nclassification report:")
print(metrics.classification_report(label_tw, y_pred,
                                    target_names=['Positive','Negative', 'Neutral']))

(ppos, pneg, pneu) = metrics.precision_score(label_tw, y_pred, average=None)
(rpos, rneg, rneu) = metrics.recall_score(label_tw, y_pred, average=None)
(fpos, fneg, fneu) = metrics.f1_score(label_tw, y_pred, average=None)
print("\n         precision  recall   f1")
print("positif    %0.3f" % ppos, "   %0.3f" % rpos, "  %0.3f" % fpos)
print("negatif    %0.3f" % pneg, "   %0.3f" % rneg, "  %0.3f" % fpos)
print("neutral    %0.3f" % pneu, "   %0.3f" % rneu, "  %0.3f" % fneu)
print("accuracy                    %0.3f \n"% metrics.accuracy_score(label_tw, y_pred))

+----+-----+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-----------+
|    |   0 |   test acc |   test pre |   test rec |   test f1  |   train acc |   train pre |   train rec |   train f1  |      time |
|----+-----+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-----------|
|  0 |   1 |   0.627238 |   0.701572 |   0.627238 |   0.561505 |    0.739928 |    0.807764 |    0.739928 |    0.715101 | 0.0220373 |
|  1 |   2 |   0.744246 |   0.807083 |   0.744246 |   0.68943  |    0.728487 |    0.798802 |    0.728487 |    0.701928 | 0.0273864 |
|  2 |   3 |   0.78133  |   0.831767 |   0.78133  |   0.752757 |    0.726071 |    0.796132 |    0.726071 |    0.697208 | 0.0208204 |
|  3 |   4 |   0.68798  |   0.753537 |   0.68798  |   0.651946 |    0.733888 |    0.799521 |    0.733888 |    0.706845 | 0.0199568 |
|  4 |   5 |   0.622123 |   0.705493 |   0.622123 |   0.563017 |    0

In [ ]:
from sklearn.model_selection import StratifiedKFold
import statistics

fold = StratifiedKFold(n_splits=10)

X = text_tw
y = label_tw

a = 1
accs = []
weigted = []
for train, test in fold.split(text_tw, label_tw):
  ypred = model.fit(X[train], y[train]).predict(X[test])
  print("Fold ", a)
  (ppos, pneg, pneu) = metrics.precision_score(y[test], ypred, average=None)
  (rpos, rneg, rneu) = metrics.recall_score(y[test], ypred, average=None)
  (fpos, fneg, fneu) = metrics.f1_score(y[test], ypred, average=None)
  weigted.append([metrics.accuracy_score(y[test], ypred), metrics.precision_score(y[test], ypred, average='weighted'),metrics.recall_score(y[test], ypred, average='weighted'),metrics.f1_score(y[test], ypred, average='weighted')])
  print("         precision  recall   f1")
  print("positif    %0.3f" % ppos, "   %0.3f" % rpos, "  %0.3f" % fpos)
  print("negatif    %0.3f" % pneg, "   %0.3f" % rneg, "  %0.3f" % fpos)
  print("neutral    %0.3f" % pneu, "   %0.3f" % rneu, "  %0.3f" % fneu)
  print("accuracy                    %0.3f \n"% metrics.accuracy_score(y[test], ypred))
  print(metrics.confusion_matrix(y[test], ypred))
  print("\n")
  a = a + 1
a = 1
print("         acc   prec   rec    f1")
wpres = []
wrecs = []
wf1ss = []
for acc, wpre, wrec, wf1s in weigted:
  accs.append(acc)
  wpres.append(wpre)
  wrecs.append(wrec)
  wf1ss.append(wf1s)
  print("Fold %2.0f" % a, "%2.3f" % acc, "%2.3f" % wpre, " %2.3f" % wrec, " %2.3f" % wf1s)
  a=a+1

print("\nstdev acc : %0.3f"% statistics.stdev(accs))
print("stdev pre : %0.3f"% statistics.stdev(wpres))
print("stdev rec : %0.3f"% statistics.stdev(wrecs))
print("stdev f1s : %0.3f"% statistics.stdev(wf1ss))

Fold  1
         precision  recall   f1
positif    0.592    0.967   0.734
negatif    0.762    0.440   0.734
neutral    0.880    0.072   0.133
accuracy                    0.627 

[[744  25   0]
 [271 215   3]
 [242  42  22]]


Fold  2
         precision  recall   f1
positif    0.676    0.987   0.803
negatif    0.913    0.769   0.803
neutral    0.967    0.095   0.173
accuracy                    0.744 

[[759  10   0]
 [112 376   1]
 [251  26  29]]


Fold  3
         precision  recall   f1
positif    0.704    0.986   0.821
negatif    0.951    0.787   0.821
neutral    0.963    0.258   0.407
accuracy                    0.781 

[[758   8   3]
 [104 385   0]
 [215  12  79]]


Fold  4
         precision  recall   f1
positif    0.631    0.953   0.759
negatif    0.838    0.573   0.759
neutral    0.926    0.206   0.337
accuracy                    0.688 

[[733  34   2]
 [206 280   3]
 [223  20  63]]


Fold  5
         precision  recall   f1
positif    0.580    0.975   0.727
negatif    0.821    0.